In [2]:
import spacy
from zss import Node, simple_distance
from functools import partial
# Imports
import numpy as np
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import spacy
from tqdm import tqdm
tqdm.pandas(desc='Processing text')
nlp = spacy.load("fr_core_news_sm")

In [3]:
import spacy
from zss import Node, simple_distance
from functools import partial

# Load the French language model
nlp = spacy.load("fr_core_news_md")

# Function to convert a spaCy token to a zss Node recursively
def token_to_zss_node(mode, token):
    if mode=="dep":
        node = Node(token.dep_)
    elif mode=="text":
        node = Node(token.orth_)
    elif mode=="pos":
        node = Node(token.pos_)
        
    for child in token.children:
        node.addkid(token_to_zss_node(mode=mode, token=child))
    return node

def build_zss_tree(spacy_doc, zss_parser):
    root = Node("ROOT")
    for sent in spacy_doc.sents:
        root.addkid(zss_parser(sent.root))
    return root

# Function to compute tree edit distance between two documents
def compute_tree_edit_distance(parsed_doc1, parsed_doc2, zss_parser):
    root1 = build_zss_tree(spacy_doc=parsed_doc1,zss_parser=zss_parser)
    root2 = build_zss_tree(spacy_doc=parsed_doc2,zss_parser=zss_parser)
    # Compute the tree edit distance using zss
    return simple_distance(root1, root2)

def _remove_stop_words(doc):
    return " ".join([str(t) for t in doc if not t.is_stop])

def semantic_similarity(doc1, doc2):
    return doc1.similarity(doc2)

# zss parser
zss_dependency_parser= partial(token_to_zss_node,'dep')

# Construct Questions Entries

In [4]:
conn = duckdb.connect(database="test.db", read_only=False)
cur = conn.cursor()

In [13]:
cur.execute("CREATE OR REPLACE TABLE questions AS SELECT * FROM './data/questions_fr_train.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_simple AS SELECT * FROM './data/questions_syn_simple_ask.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_simple_tc AS SELECT * FROM './data/questions_syn_simple_ask_qcc.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_simple_tc_fewshots AS SELECT * FROM './data/questions_syn_simple_ask_qcc_fewshots.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe AS SELECT * FROM './data/questions_syn_describe_then_ask.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_tc AS SELECT * FROM './data/questions_syn_describe_then_ask_qcc.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_fewshots AS SELECT * FROM './data/questions_syn_describe_then_ask_fewshots.csv';")
cur.execute("CREATE OR REPLACE TABLE questions_describe_tc_fewshots AS SELECT * FROM './data/questions_syn_describe_then_ask_qcc_fewshots.csv';")

In [49]:
dfQ = cur.execute("""
SELECT questions.article_ids, question
FROM questions
""").df()

dfQ['article_ids'] = dfQ['article_ids'].str.split(',')
dfQ = dfQ.explode('article_ids')

cur.execute("""
CREATE OR REPLACE TABLE questions AS 
SELECT * FROM dfQ
""")

cur.execute("""
SELECT * FROM questions
""").df()

,article_ids,question
0,22225,Je suis travailleur salarié(e). Puis-je refuse...
1,22226,Je suis travailleur salarié(e). Puis-je refuse...
2,22227,Je suis travailleur salarié(e). Puis-je refuse...
3,22228,Je suis travailleur salarié(e). Puis-je refuse...
4,22229,Je suis travailleur salarié(e). Puis-je refuse...
...,...,...
5779,14479,Si je ne rembourse pas mon crédit hypothécaire...
5780,14480,Si je ne rembourse pas mon crédit hypothécaire...
5781,14481,Si je ne rembourse pas mon crédit hypothécaire...
5782,14482,Si je ne rembourse pas mon crédit hypothécaire...


In [50]:
cur.execute("""
    CREATE OR REPLACE TABLE questions_summary as 
    (SELECT questions.article_ids, question, 
    questions_simple.synthetic_question as questions_simple, 
    questions_simple_tc.synthetic_question as questions_simple_tc, 
    questions_simple_tc_fewshots.synthetic_question as questions_simple_tc_fewshots,
    questions_describe.synthetic_question as questions_describe, 
    questions_describe_tc.synthetic_question as questions_describe_tc, 
    questions_describe_fewshots.synthetic_question as questions_describe_fewshots, 
    questions_describe_tc_fewshots.synthetic_question as questions_describe_tc_fewshots
    FROM questions, questions_simple, questions_simple_tc, questions_simple_tc_fewshots, questions_describe, questions_describe_tc, questions_describe_fewshots, questions_describe_tc_fewshots
    where questions.article_ids not like '%,%'
    and questions.article_ids = questions_simple.article_ids
    and questions.article_ids = questions_simple_tc.article_ids
    and questions.article_ids = questions_simple_tc_fewshots.article_ids
    and questions.article_ids = questions_describe.article_ids
    and questions.article_ids = questions_describe_tc.article_ids
    and questions.article_ids = questions_describe_fewshots.article_ids
    and questions.article_ids = questions_describe_tc_fewshots.article_ids)
""")

In [131]:
dfQ_summary = cur.execute("""
    SELECT *
    FROM questions_summary
""").df()
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots
0,1115,Les parents ne sont pas mariés ensemble. La mè...,Quels sont les modes de preuve de la filiation...,Quel est le critère principal pour prouver la ...,La coparente prétendue peut-elle prouver sa fi...,Une personne revendique la filiation à l'égard...,Un couple a recours à un service de médiation ...,Une personne prétend être le parent d'un enfan...,La filiation d'un enfant est contestée par une...
1,4462,Quels recours exercer contre les abus lors d'u...,Quel est l'organe chargé de représenter l'ordr...,Est-ce que le Conseil d'administration représe...,Est-ce que le Conseil d'administration représe...,Le Conseil d'administration d'un ordre profess...,Un consommateur a un litige avec une entrepris...,Un avocat agit au nom de son ordre professionn...,Un avocat souhaite engager des poursuites judi...
2,1823,Puis-je faire une donation de tous mes biens à...,Quelles sont les conditions pour que la quotit...,Quels sont les bénéficiaires qui peuvent recev...,La disposition de la quotité disponible aux en...,Un parent souhaite organiser sa succession en ...,Un consommateur belge a un litige avec un four...,Un parent souhaite donner une part de son patr...,Un individu souhaite faire une donation à ses ...
3,1064,Qui doit payer les frais de dernière maladie ?,Quels critères sont pris en compte pour déterm...,Combien de critères sont pris en compte pour a...,Les aliments peuvent-ils être accordés de mani...,Un particulier a intenté une action en justice...,Un individu a entamé une procédure d'arbitrage...,Deux époux sont en instance de divorce et l'un...,Un individu réclame des aliments à une autre p...
4,4768,Quels documents dois-je remettre au bureau d'a...,Quelle autorité est chargée de prévenir le gre...,Quand le greffier notifie-t-il l'administratio...,"Lorsqu'une décision d'assistance est accordée,...",Dans le cadre d'une affaire judiciaire impliqu...,Un consommateur a déposé une demande de règlem...,"J'ai obtenu une décision d'assistance, suite à...",Une personne a demandé une assistance sous cer...
...,...,...,...,...,...,...,...,...,...
4590,14471,Qu'est-ce qu'un crédit hypothécaire ?,Quelles informations doit communiquer le prête...,Est-ce que le prêteur est légalement tenu de c...,"Lors de mon refus de crédit, le prêteur est-il...",Un consommateur a récemment essuyé un refus de...,Un consommateur a vu sa demande de crédit refu...,Je viens de me voir refuser un crédit. Quelles...,Je viens de me voir refuser un crédit par une ...
4591,1381,Peut-on changer d'administrateur de biens et/o...,Dans quels cas le juge de paix est-il autorisé...,Dans quelles circonstances le juge de paix peu...,Peut-on demander au juge de paix de modifier l...,Un individu a été placé sous protection juridi...,Dans le cadre d'une situation impliquant une p...,Une personne qui a été placée sous administrat...,Une personne sous protection légale souhaite m...
4592,1381,Les régimes de protection des personnes fragil...,Dans quels cas le juge de paix est-il autorisé...,Dans quelles circonstances le juge de paix peu...,Peut-on demander au juge de paix de modifier l...,Un individu a été placé sous protection juridi...,Dans le cadre d'une situation impliquant une p...,Une personne qui a été placée sous administrat...,Une personne sous protection légale souhaite m...
4593,5556,Qu'est-ce qu'une procédure sommaire d'injoncti...,Quelles sont les informations obligatoires dev...,Est-ce que la requête doit être adressée au ju...,À quel moment et de quelle manière doit être a...,Un requérant souhaite adresser une demande au ...,Un consommateur a eu un litige avec une entrep...,J'ai une réclamation à déposer devant un tribu...,Un demandeur souhaite introduire une requête e...


In [132]:
dfQ_summary = dfQ_summary.sample(n=2000,random_state=42)

In [133]:
dfQ_summary['simple_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple']),zss_parser=zss_dependency_parser), axis=1)

In [134]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple_tc_fewshots']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['simple_tc_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_simple_tc']),zss_parser=zss_dependency_parser), axis=1)

In [135]:
dfQ_summary['describe_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['describe_tc_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_tc']),zss_parser=zss_dependency_parser), axis=1)

In [136]:
dfQ_summary['describe_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_fewshots']),zss_parser=zss_dependency_parser), axis=1)

dfQ_summary['describe_tc_fewshots_tree_edit_distance'] = dfQ_summary.apply(lambda row: compute_tree_edit_distance(nlp(row['question']), nlp(row['questions_describe_tc_fewshots']),zss_parser=zss_dependency_parser), axis=1)

In [137]:
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots,simple_tree_edit_distance,simple_tc_fewshots_tree_edit_distance,simple_tc_tree_edit_distance,describe_tree_edit_distance,describe_tc_tree_edit_distance,describe_fewshots_tree_edit_distance,describe_tc_fewshots_tree_edit_distance
3201,6517,"Je me suis bagarré à la fin d’une soirée, qu’e...",Quelles sont les peines prévues pour une perso...,Quelle est la peine encourue si les coups ou l...,Les coups ou blessures causant une maladie ou ...,Un individu a été agressé et a subi des blessu...,"Une personne a agressé physiquement une autre,...",Une dispute familiale a conduit à des coups et...,Après avoir reçu des coups d'une tierce person...,24.0,22.0,19.0,32.0,32.0,33.0,41.0
2820,4767,Puis-je avoir l'assistance gratuite d'un avoca...,La question pourrait être:\n\nQuels frais la p...,Quelle est la procédure pour obtenir l'expédit...,Les décisions rendues par le tribunal sont-ell...,Un particulier a intenté une action en justice...,Un consommateur a entamé une procédure de règl...,Une décision a été rendue par un tribunal. Est...,"Après avoir obtenu une décision, la partie req...",29.0,23.0,18.0,56.0,32.0,29.0,26.0
1616,1282,Les père et mère de l'enfant résident ensemble...,Quels droits a le parent qui n'exerce pas conj...,Est-ce que chacun des parents est réputé agir ...,L'un des parents décide seul d'effectuer une t...,Deux parents exercent conjointement l'autorité...,Deux parents exercent conjointement l'autorité...,Les parents exercent conjointement l'autorité ...,Les parents de l'enfant ne vivent plus ensembl...,33.0,31.0,34.0,50.0,76.0,23.0,36.0
2696,2845,Puis-je invoquer la prescription pour ne pas p...,Quel est le délai de prescription pour l'actio...,Quand débute le délai de prescription pour l'a...,Les prestataires de soins ont fourni des servi...,Un médecin a fourni des prestations médicales ...,Un patient a reçu des prestations médicales d'...,Un patient a reçu des services médicaux d'un p...,J'ai reçu des prestations médicales en mars 20...,27.0,30.0,15.0,68.0,63.0,29.0,26.0
3488,4480,Ai-je droit à l'aide juridique (ex pro deo) si...,Quels sont les arrondissements judiciaires où ...,Est-ce que la Commission d'aide juridique fran...,Comment est composée la Commission d'aide juri...,Dans le cadre de l'accès à la justice en Belgi...,Dans le cadre du règlement extrajudiciaire des...,Je suis confronté à des difficultés juridiques...,Je suis confronté à des problèmes juridiques m...,16.0,15.0,20.0,45.0,73.0,35.0,37.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,1276,Que faire si je ne peux pas me mettre d'accord...,Quels sont les recours disponibles pour un par...,Est-ce que chaque parent est réputé agir avec ...,Les parents ne vivent pas ensemble. Doivent-il...,Deux parents vivent ensemble mais se trouvent ...,Un couple non marié a un enfant ensemble. Ils ...,Les parents vivent ensemble mais n'arrivent pa...,Deux parents vivent ensemble et exercent conjo...,19.0,41.0,33.0,47.0,55.0,45.0,42.0
3058,4506,Quels sont les revenus pris en compte pour l'a...,Quels sont les pourcentages maximaux prévus po...,La décision du bureau d'aide juridique concern...,Un avocat a permis à son client de percevoir u...,Un avocat a réussi à obtenir des sommes d'arge...,Un avocat a aidé un bénéficiaire à percevoir d...,J'ai obtenu un règlement grâce à l'interventio...,Un avocat a aidé un client à obtenir des somme...,19.0,45.0,22.0,76.0,40.0,38.0,28.0
2905,1334,Qui contrôle le fonctionnement de la tutelle ?,À partir de quelle date les intérêts commencen...,Quand les intérêts relatifs au reliquat dû par...,Le reliquat dû par le tuteur à partir du momen...,Contexte: Le tuteur légal d'un mineur a pris e...,Un mineur sous tutelle a des montants dus à so...,Je suis tuteur légal d'un mineur. Le compte de...,Un mineur a de l'argent placé sous la

In [138]:
dfQ_summary['simple_tree_edit_distance'].mean()

22.7875

In [139]:
dfQ_summary['simple_tc_tree_edit_distance'].mean()

23.953

In [140]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance'].mean()

25.211

In [141]:
dfQ_summary['describe_tree_edit_distance'].mean()

55.5865

In [142]:
dfQ_summary['describe_tc_tree_edit_distance'].mean()

50.4745

In [143]:
dfQ_summary['describe_fewshots_tree_edit_distance'].mean()

32.899

In [144]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance'].mean()

32.9725

# semantic 

In [86]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple_tc_fewshots'])))), axis=1)

dfQ_summary['simple_tc_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple_tc'])))), axis=1)

In [87]:
dfQ_summary['simple_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_simple'])))), axis=1)

In [70]:
dfQ_summary['describe_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe'])))), axis=1)

dfQ_summary['describe_tc_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_tc'])))), axis=1)

dfQ_summary['describe_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_fewshots'])))), axis=1)

dfQ_summary['describe_tc_fewshots_semantic_similarity'] = dfQ_summary.apply(lambda row: semantic_similarity(nlp(_remove_stop_words(nlp(row['question']))), nlp(_remove_stop_words(nlp(row['questions_describe_tc_fewshots'])))), axis=1)

In [71]:
dfQ_summary['describe_semantic_similarity'].mean()

0.5535614752042263

In [72]:
dfQ_summary['describe_tc_semantic_similarity'].mean()

0.5656643756623633

In [73]:
dfQ_summary['describe_fewshots_semantic_similarity'].mean()

0.603698511885458

In [74]:
dfQ_summary['describe_tc_fewshots_semantic_similarity'].mean()

0.6085111030132824

In [81]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'].mean()

0.5915320833611377

In [89]:
dfQ_summary['simple_semantic_similarity'].mean()

0.5464909524748509

In [90]:
dfQ_summary['simple_tc_semantic_similarity'].mean()

0.576864490860546

In [93]:
dfQ_summary['simple_tc_fewshots_semantic_similarity'].mean()

0.5915320833611377

# sentence length difference normalization

In [145]:
dfQ_summary['simple_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [146]:
dfQ_summary['simple_tc_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple_tc'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [147]:
dfQ_summary['simple_tc_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_simple_tc_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [148]:
dfQ_summary['describe_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [149]:
dfQ_summary['describe_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [150]:
dfQ_summary['describe_tc_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_tc'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [151]:
dfQ_summary['describe_tc_fewshots_length_diff'] = dfQ_summary.apply(lambda row: len(row['questions_describe_tc_fewshots'].split(" ")) - len(row['question'].split(" ")), axis=1)

In [152]:
dfQ_summary

,article_ids,question,questions_simple,questions_simple_tc,questions_simple_tc_fewshots,questions_describe,questions_describe_tc,questions_describe_fewshots,questions_describe_tc_fewshots,simple_tree_edit_distance,...,describe_tc_tree_edit_distance,describe_fewshots_tree_edit_distance,describe_tc_fewshots_tree_edit_distance,simple_length_diff,simple_tc_length_diff,simple_tc_fewshots_length_diff,describe_length_diff,describe_fewshots_length_diff,describe_tc_length_diff,describe_tc_fewshots_length_diff
3201,6517,"Je me suis bagarré à la fin d’une soirée, qu’e...",Quelles sont les peines prévues pour une perso...,Quelle est la peine encourue si les coups ou l...,Les coups ou blessures causant une maladie ou ...,Un individu a été agressé et a subi des blessu...,"Une personne a agressé physiquement une autre,...",Une dispute familiale a conduit à des coups et...,Après avoir reçu des coups d'une tierce person...,24.0,...,32.0,33.0,41.0,11,6,4,18,22,16,27
2820,4767,Puis-je avoir l'assistance gratuite d'un avoca...,La question pourrait être:\n\nQuels frais la p...,Quelle est la procédure pour obtenir l'expédit...,Les décisions rendues par le tribunal sont-ell...,Un particulier a intenté une action en justice...,Un consommateur a entamé une procédure de règl...,Une décision a été rendue par un tribunal. Est...,"Après avoir obtenu une décision, la partie req...",29.0,...,32.0,29.0,26.0,8,-4,2,46,11,20,8
1616,1282,Les père et mère de l'enfant résident ensemble...,Quels droits a le parent qui n'exerce pas conj...,Est-ce que chacun des parents est réputé agir ...,L'un des parents décide seul d'effectuer une t...,Deux parents exercent conjointement l'autorité...,Deux parents exercent conjointement l'autorité...,Les parents exercent conjointement l'autorité ...,Les parents de l'enfant ne vivent plus ensembl...,33.0,...,76.0,23.0,36.0,9,8,10,31,6,54,21
2696,2845,Puis-je invoquer la prescription pour ne pas p...,Quel est le délai de prescription pour l'actio...,Quand débute le délai de prescription pour l'a...,Les prestataires de soins ont fourni des servi...,Un médecin a fourni des prestations médicales ...,Un patient a reçu des prestations médicales d'...,Un patient a reçu des services médicaux d'un p...,J'ai reçu des prestations médicales en mars 20...,27.0,...,63.0,29.0,26.0,16,6,22,58,20,52,15
3488,4480,Ai-je droit à l'aide juridique (ex pro deo) si...,Quels sont les arrondissements judiciaires où ...,Est-ce que la Commission d'aide juridique fran...,Comment est composée la Commission d'aide juri...,Dans le cadre de l'accès à la justice en Belgi...,Dans le cadre du règlement extrajudiciaire des...,Je suis confronté à des difficultés juridiques...,Je suis confronté à des problèmes juridiques m...,16.0,...,73.0,35.0,37.0,0,7,1,32,20,58,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,1276,Que faire si je ne peux pas me mettre d'accord...,Quels sont les recours disponibles pour un par...,Est-ce que chaque parent est réputé agir avec ...,Les parents ne vivent pas ensemble. Doivent-il...,Deux parents vivent ensemble mais se trouvent ...,Un couple non marié a un enfant ensemble. Ils ...,Les parents vivent ensemble mais n'arrivent pa...,Deux parents vivent ensemble et exercent conjo...,19.0,...,55.0,45.0,42.0,-4,9,15,29,25,33,19
3058,4506,Quels sont les revenus pris en compte pour l'a...,Quels sont les pourcentages maximaux prévus po...,La décision du bureau d'aide juridique concern...,Un avocat a permis à son client de percevoir u...,Un avocat a réussi à obtenir des sommes d'arge...,Un avocat a aidé un bénéficiaire à percevoir d...,J'ai obtenu un règlement grâce à l'interventio...,Un avocat a aidé un client à obtenir des somme...,19.0,...,40.0,38.0,28.0,6,5,25,59,21,21,13
2905,1334,Qui contrôle le fonctionnement de la tutelle ?,À partir de quelle date les intérêts commencen...,Quand les intérêts relatifs au reliquat dû par...,Le reliquat dû par le tuteur à partir du momen...,Contexte: Le tuteur lég

In [153]:
dfQ_summary['simple_tree_edit_distance_norm'] = dfQ_summary['simple_tree_edit_distance'] - dfQ_summary['simple_length_diff']

In [154]:
dfQ_summary['simple_tc_tree_edit_distance_norm'] = dfQ_summary['simple_tc_tree_edit_distance'] - dfQ_summary['simple_tc_length_diff']

In [155]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance_norm'] = dfQ_summary['simple_tc_fewshots_tree_edit_distance'] - dfQ_summary['simple_tc_fewshots_length_diff']

In [156]:
dfQ_summary['describe_tree_edit_distance_norm'] = dfQ_summary['describe_tree_edit_distance'] - dfQ_summary[
    'describe_length_diff']

In [157]:
dfQ_summary['describe_fewshots_tree_edit_distance_norm'] = dfQ_summary['describe_fewshots_tree_edit_distance'] - dfQ_summary['describe_fewshots_length_diff']

In [158]:
dfQ_summary['describe_tc_tree_edit_distance_norm'] = dfQ_summary['describe_tc_tree_edit_distance'] - dfQ_summary['describe_tc_length_diff']

In [159]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance_norm'] = dfQ_summary['describe_tc_fewshots_tree_edit_distance'] - dfQ_summary['describe_tc_fewshots_length_diff']

In [160]:
dfQ_summary['simple_tree_edit_distance_norm'].mean()

12.938

In [161]:
dfQ_summary['simple_tc_tree_edit_distance_norm'].mean()

13.8785

In [162]:
dfQ_summary['simple_tc_fewshots_tree_edit_distance_norm'].mean()

13.0885

In [163]:
dfQ_summary['describe_tree_edit_distance_norm'].mean()

12.442

In [164]:
dfQ_summary['describe_fewshots_tree_edit_distance_norm'].mean()

11.669

In [165]:
dfQ_summary['describe_tc_tree_edit_distance_norm'].mean()

13.208

In [166]:
dfQ_summary['describe_tc_fewshots_tree_edit_distance_norm'].mean()

11.9575